In [139]:
import ee
ee.Authenticate()
ee.Initialize()
import LEAF
import SL2PV0 
import SL2PV1

In [140]:
import os, pickle
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import itertools

In [141]:
nlcd='deciduousForest'
variable="Surface_Reflectance"
sensor='S2'
datapath='C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LS_S2_fusion_check/data/%s/%s/'%(variable,sensor)
print(datapath)

C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LS_S2_fusion_check/data/Surface_Reflectance/S2/


In [142]:
siteList=["projects/ee-modis250/assets/validationGBOV/NEON_2013-22_RMs_updated_parsed"]

In [143]:
input=siteList[0]
sampleRecords =  ee.FeatureCollection(input).sort('system:time_start', False).map(lambda feature: feature.set('timeStart',feature.get('system:time_start')))
sampleRecords =  sampleRecords.toList(sampleRecords.size()).getInfo()

In [144]:
ESUs=[ss ['properties']['PLOT_ID'] for ss in sampleRecords ]

In [145]:
# ESUs=['ABBY_067', 'ABBY_068', 'ABBY_069', 'BARR_052', 'BARR_056',       'BARR_072', 'BART_034', 'BART_041', 'BART_047', 'BLAN_034',       'BLAN_048', 'BLAN_052', 'BONA_070', 'BONA_080', 'BONA_082',       'BONA_084', 'BONA_089', 'CLBJ_001', 'CLBJ_002', 'CLBJ_003',
#        'CLBJ_021', 'CLBJ_029', 'CPER_001', 'CPER_002', 'CPER_003',       'CPER_004', 'CPER_005', 'CPER_006', 'CPER_007', 'CPER_008',       'CPER_009', 'CPER_010', 'CPER_011', 'CPER_012', 'CPER_013',       'CPER_014', 'CPER_015', 'CPER_016', 'CPER_017', 'CPER_018',       'CPER_019', 'CPER_020', 'CPER_046', 'CPER_056', 'CPER_062',       'DCFS_069', 'DCFS_081', 'DCFS_090', 'DEJU_052', 'DEJU_056',
#        'DEJU_061', 'DELA_044', 'DELA_046', 'DELA_051', 'DELA_053',       'DSNY_053', 'DSNY_061', 'DSNY_066', 'GRSM_064', 'GRSM_065',       'GRSM_066', 'GUAN_052', 'GUAN_054', 'GUAN_055', 'HARV_041',       'HARV_049', 'HARV_050', 'HEAL_048', 'HEAL_063', 'HEAL_070',
#        'JERC_047', 'JERC_050', 'JERC_054', 'JERC_060', 'JERC_062',       'JORN_053', 'JORN_055', 'JORN_061', 'KONA_070', 'KONA_071',       'KONA_074', 'KONZ_042', 'KONZ_050', 'KONZ_051', 'KONZ_068',       'LAJA_047', 'LAJA_054', 'LAJA_065', 'LENO_063', 'LENO_064',
#        'LENO_067', 'MLBS_069', 'MLBS_071', 'MLBS_074', 'MOAB_049',       'MOAB_061', 'MOAB_070', 'NIWO_048', 'NIWO_054', 'NIWO_056',       'NIWO_060', 'NOGP_072', 'NOGP_073', 'NOGP_087', 'OAES_042',       'OAES_043', 'OAES_044', 'ONAQ_051', 'ONAQ_060', 'ONAQ_069',
#        'ORNL_001', 'ORNL_002', 'ORNL_003', 'ORNL_004', 'ORNL_006',       'ORNL_007', 'ORNL_008', 'ORNL_009', 'ORNL_010', 'ORNL_012',       'ORNL_014', 'ORNL_021', 'ORNL_027', 'ORNL_029', 'ORNL_045',       'ORNL_050', 'ORNL_054', 'OSBS_025', 'OSBS_030', 'OSBS_038',
#        'RMNP_045', 'RMNP_046',       'RMNP_047', 'SCBI_049', 'SCBI_062', 'SCBI_063', 'SERC_003',       'SERC_005', 'SERC_010', 'SERC_011', 'SERC_016', 'SERC_052',       'SERC_053', 'SERC_054', 'SERC_061', 'SERC_062', 'SJER_049',
#        'SJER_055', 'SJER_057', 'SOAP_047', 'SOAP_054', 'SOAP_058',       'SRER_046', 'SRER_048', 'SRER_057', 'STEI_046', 'STEI_055',       'STEI_058', 'STER_008', 'STER_009', 'STER_014', 'TALL_046',       'TALL_048', 'TALL_049', 'TEAK_046', 'TEAK_047', 'TEAK_057',
#        'TOOL_041', 'TOOL_042', 'TOOL_043', 'TREE_053', 'TREE_055',       'TREE_056', 'UKFS_047', 'UKFS_051', 'UKFS_053', 'UKFS_058',       'UNDE_044', 'UNDE_045', 'UNDE_048', 'UNDE_053', 'UNDE_057',       'UNDE_059', 'WOOD_044', 'WOOD_058', 'WOOD_064', 'WOOD_069',
#        'WREF_070', 'WREF_074', 'WREF_078', 'WREF_081', 'WREF_082',       'WREF_086', 'WREF_088', 'YELL_050', 'YELL_057', 'YELL_060'] #'PUUM_031', 'PUUM_039', 'PUUM_041',

In [146]:
1

1

In [147]:
exist_files=[ff for ff in os.listdir(datapath) if ff.endswith('.pkl') and nlcd in ff and variable in ff]
exist_data=[]
for ff in exist_files:
    with open(os.path.join(datapath,ff), "rb") as fp:   #Pickling
        data = pickle.load(fp)
    exist_data=exist_data+list(data.keys())
print(exist_files,len(np.unique(exist_data)))

[] 0


In [148]:
all_sites=[ss ['properties']['PLOT_ID'] for ss in sampleRecords ]
nlcd_sites=np.unique([ss ['properties']['PLOT_ID'] for ss in sampleRecords if ss ['properties']['nlcd']==nlcd])
add_ESUs=np.unique([esu for esu in ESUs if esu not in exist_data and esu in nlcd_sites])
print(len(all_sites),len(nlcd_sites),len(exist_data),len(add_ESUs))

14141 249 0 249


In [149]:
plots_idx=[all_sites.index(ss) for ss in add_ESUs]
len(plots_idx)

249

In [150]:
plots_idx

[6342,
 5882,
 5419,
 163,
 4926,
 4455,
 781,
 53,
 12352,
 51,
 11421,
 11429,
 11422,
 11423,
 10917,
 10910,
 10913,
 11424,
 10930,
 20,
 149,
 515,
 1873,
 6332,
 1877,
 108,
 124,
 32,
 557,
 558,
 241,
 240,
 239,
 361,
 365,
 4204,
 6078,
 4200,
 8375,
 6082,
 5629,
 6997,
 4208,
 8377,
 4194,
 345,
 82,
 96,
 103,
 8601,
 238,
 220,
 8587,
 5599,
 3560,
 715,
 1167,
 722,
 7321,
 676,
 221,
 3564,
 2864,
 669,
 8586,
 672,
 3557,
 178,
 184,
 182,
 11903,
 11504,
 11006,
 11914,
 11913,
 11005,
 4872,
 5377,
 7680,
 4875,
 5378,
 9009,
 4410,
 1037,
 4056,
 7010,
 305,
 294,
 7009,
 313,
 7011,
 8683,
 6,
 3,
 1604,
 2027,
 5558,
 8249,
 11160,
 11626,
 11630,
 8240,
 9185,
 5554,
 3507,
 1594,
 11624,
 12099,
 12094,
 12093,
 1614,
 8246,
 12091,
 11621,
 68,
 79,
 65,
 2318,
 4039,
 4031,
 1915,
 3103,
 1467,
 1122,
 3106,
 1117,
 648,
 3104,
 5590,
 2813,
 10650,
 1123,
 60,
 602,
 599,
 600,
 59,
 9235,
 6904,
 902,
 3896,
 11420,
 10348,
 168,
 4017,
 13412,
 6414,
 6415

In [151]:
dict={'HLS':{'collection':'NASA/HLS/HLSL30/v002','scale':30,'buffer':45},'LC08':{'collection':'LANDSAT/LC08/C02/T1_L2','scale':30,'buffer':45},'S2':{'collection':'COPERNICUS/S2_SR_HARMONIZED','scale':20,'buffer':30}}

In [152]:
dict[sensor]

{'collection': 'COPERNICUS/S2_SR_HARMONIZED', 'scale': 20, 'buffer': 30}

In [ ]:
data = LEAF.sampleTimeSeries(siteList, \
                                                imageCollectionName=dict[sensor]['collection'],\
                                                algorithm=SL2PV1, 
                                                variableName=variable,\
                                                maxCloudcover=90,\
                                                outputScaleSize=dict[sensor]['scale'],\
                                                inputScaleSize=dict[sensor]['scale'],\
                                                bufferSpatialSize = dict[sensor]['buffer'],\
                                                bufferTemporalSize =["2018-01-01","2023-12-31"],  \
                                                subsamplingFraction = 0.99,\
                                                outputFileName=datapath,\
                                                nlcd=nlcd,\
                                                plots=plots_idx)

STARTING LEAF IMAGE for  COPERNICUS/S2_SR_HARMONIZED
BART_002
BART_003
BART_006
BART_010
BART_012
BART_013
BART_036
BART_041
BART_046
BART_047
BLAN_001
BLAN_007
BLAN_009
BLAN_011
BLAN_012


In [84]:
1

1